In [9]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from sqlalchemy import create_engine, text
from typing import List

In [10]:
postgres_config = {
    "host": "findy-medium-stage.czmgcqkw4ett.ap-southeast-1.rds.amazonaws.com",
    "database": "findy_medium_stage",
    "user": "postgres",
    "password": "F!nDy!Med!umStage2o24",
    "port": "5432"
}

In [11]:
def build_upsert_query(cols: List[str],
                       table_name: str,
                       unique_key: List[str]=[],
                       cols_not_for_update: List[str] = None) -> str:
    """
    Builds postgres upsert query using input arguments.
    Note: In the absence of unique_key, this will be just an insert query.
    Example : build_upsert_query(
        ['col1', 'col2', 'col3', 'col4'],
        "my_table",
        ['col1'],
        ['col2']
    ) ->
    INSERT INTO my_table (col1, col2, col3, col4) VALUES %s
    ON CONFLICT (col1) DO UPDATE SET (col3, col4) = (EXCLUDED.col3, EXCLUDED.col4) ;
    :param cols: the postgres table columns required in the
        insert part of the query.
    :param table_name: the postgres table name.
    :param unique_key: unique_key of the postgres table for checking
        unique constraint violations.
    :param cols_not_for_update: columns in cols which are not required in
        the update part of upsert query.
    :return: Upsert query as per input arguments.
    """
    cols = [f'"{col}"' for col in cols]
    cols_str = ', '.join(cols)
    insert_query = """ INSERT INTO %s (%s) VALUES %%s """ % (
        table_name, cols_str
    )
    if cols_not_for_update is not None:
        cols_not_for_update.extend(unique_key)
    else:
        cols_not_for_update = [col for col in unique_key]
    cols_not_for_update = [f'"{col}"' for col in cols_not_for_update]
    unique_key = [f'"{col}"' for col in unique_key]
    unique_key_str = ', '.join(unique_key)

    update_cols = [f"{col}" for col in cols if col not in cols_not_for_update]
    update_cols_str = ', '.join(update_cols)
    update_cols_with_excluded_markers = [f'EXCLUDED.{col}' for col in update_cols]
    update_cols_with_excluded_markers_str = ', '.join(
        update_cols_with_excluded_markers
    )
    if len(update_cols) > 1:
        equality_clause = "(%s) = (%s)"
    else:
        equality_clause = "%s = %s"

    on_conflict_clause = f""" ON CONFLICT (%s) DO UPDATE SET {equality_clause} ;"""
    on_conflict_clause = on_conflict_clause % (
        unique_key_str,
        update_cols_str,
        update_cols_with_excluded_markers_str
    )
    if len(unique_key) == 0:
        return insert_query
    return insert_query + on_conflict_clause

In [12]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_influencer"
bucket_name = "3_staging_area"
api_name = "search_for_author_square"

In [ ]:
processing_blobs = [
{
    "blob": blob,
    "date": blob.name.split('/')[2],
    "batch": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs(bucket_name,prefix="1_xingtu/" + table_name + "/") if blob.name.endswith(".parquet") and api_name in blob.name]
processing_blobs

In [ ]:
bucket = storage_client.get_bucket(bucket_name)
meta_blob = bucket.blob("1_xingtu/" + table_name + "/meta.json")
if meta_blob.exists():
    processed_blobs = json.loads(meta_blob.download_as_string())
else:
    processed_blobs = []
processed_blobs

In [ ]:
to_process = []
for processing_blob in processing_blobs: 
    processing_date = datetime.strptime(processing_blob["date"], "%Y-%m-%d")
    if processing_date >= datetime(2024, 9, 8):
        if processing_blob["batch"] not in [processed_blob["batch"] for processed_blob in processed_blobs if api_name in processed_blob["file_path"] and processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

In [ ]:
for item in to_process:
    engine = create_engine("postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(**postgres_config))
    l_df = pd.read_sql_table("douyin_influencer",con=engine)
    r_df = pd.read_parquet("gs://" + bucket_name + "/" + item["blob"].name)
    r_df["deleted"] = False
    df = pd.merge(l_df, r_df, on="user_id", how="right")
    df.columns = [column.replace("_y", "") for column in df.columns]
    upsert_df = df[["is_star"] + list(r_df.columns)]
    upsert_df["is_star"] = True
    upsert_df = upsert_df.dropna(subset=["core_user_id"])
    upsert_df = upsert_df[~upsert_df["tags_ids_level_two"].str.contains("10010")]
    upsert_df = upsert_df.drop_duplicates()
    upsert_df = upsert_df.drop_duplicates("core_user_id")
    upsert_df["updated"] = str(today)
    upsert_df = upsert_df[upsert_df.user_id != "6870159673392103437"]
    upsert_df.nickname = upsert_df.nickname.str.replace("'", "''")
    upsert_df.nickname = upsert_df.nickname.str.replace('"', "")
    upsert_df.nickname = upsert_df.nickname.str.replace(":", "\:")
    upsert_df.city = upsert_df.city.str.replace("'", "''")
    query = build_upsert_query(upsert_df.columns, "douyin_influencer", ["core_user_id"])
    value = ', '.join([str(record).replace('"', "'").replace("None", "NULL") for record in upsert_df.to_records(index=False)])
    query = query % value
    with engine.connect() as conn:
        result = conn.execute(text(query))
        conn.commit()
    processed_blobs.append({
        'file_path': "gs://" + bucket_name + "/" + item["blob"].name,
        'date': item["date"],
        'batch': item["batch"]
    })
    meta_blob.upload_from_string(json.dumps(processed_blobs))